In [1]:
from Bio import SeqIO
import os
from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize
import nltk

In [2]:
def getFastaData(fasta_path):
    fasta_file = fasta_path

    sequences = []
    for record in SeqIO.parse(fasta_file, "fasta"):
        sequences.append({
            "id": record.id,
            "sequence": str(record.seq)
        })

    result = []
    for sequence in sequences:

        result.append(sequence["sequence"])


    return result

In [22]:
p = getFastaData(r'')
n = getFastaData(r'')
p_label = [1 for i in range(len(p))]
n_label = [0 for i in range(len(n))]
feature = p + n
label = p_label + n_label

In [7]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from transformers import BertTokenizer, AutoTokenizer,BertModel, AutoModel 
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
class myModel(torch.nn.Module):
    def __init__(self,esm2):
        super(myModel,self).__init__()
        self.esm2 = esm2
        self.fc1 = torch.nn.Linear(640,64)
        self.fc2 = torch.nn.Linear(64,1)
        
        self.fc31 = torch.nn.Linear(640,128)
        self.fc32 = torch.nn.Linear(128,64)
        
        self.fc41 = torch.nn.Linear(640,32)
        self.fc42 = torch.nn.Linear(32,64)
        
        
        self.sigmoid = nn.Sigmoid()
        self.relu = nn.ReLU()
        self.bn1 = torch.nn.BatchNorm1d(64)
        self.dropout1 = nn.Dropout(p=0.1)
        self.dropout2 = nn.Dropout(p=0.4)
    def forward(self,x):
        outputs_ = self.esm2(**x)

        x = outputs_.last_hidden_state[:, 0, :]
        x = self.dropout1(x)
        x1 = self.fc1(x)
        x1 = self.relu(x1)
        x1 = self.dropout2(x1)
        
        x2 = self.fc31(x)
        x2 = self.relu(x2)
        x2 = self.fc32(x2)
        x2 = self.relu(x2)
        x2 = self.dropout2(x2)
        
        x3 = self.fc41(x)
        x3 = self.relu(x3)
        x3 = self.fc42(x3)
        x3 = self.relu(x3)
        x3 = self.dropout2(x3)
        
        x = x1 + x2 + x3
        
        x = self.fc2(x)
        x = self.sigmoid(x)
        return x
        
        

In [9]:
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, matthews_corrcoef, recall_score, precision_score
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_classification
from sklearn.linear_model import LogisticRegression

def comp_result(y_test, y_pred, y_proba):
    accuracy = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    auc = roc_auc_score(y_test, y_proba)
    mcc = matthews_corrcoef(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)

    print(f"Accuracy: {accuracy:.4f}")
    print(f"F1 Score: {f1:.4f}")
    print(f"AUC: {auc:.4f}")
    print(f"MCC: {mcc:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"Precision: {precision:.4f}")
    
    return accuracy,f1,auc,mcc,recall,precision

In [ ]:
from sklearn.model_selection import KFold
import numpy as np
import copy as cp

feature = np.array(feature)
label = np.array(label)
kf = KFold(n_splits=5, shuffle=True, random_state=42)
kf_in = 0
for train_index, test_index in kf.split(feature):
    save_model = None
    best_accuracy = 0
    best_f1 = 0
    best_auc = 0
    best_mcc = 0
    best_recall = 0
    best_precision = 0
    print(train_index)
    kf_in += 1
    print('kf',kf_in)
    X_train, X_test = list(feature[train_index]), list(feature[test_index])
    y_train, y_test = list(label[train_index]), list(label[test_index])

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    cache_directory = ""


    tokenizer_ = AutoTokenizer.from_pretrained("facebook/esm2_t30_150M_UR50D", cache_dir=cache_directory)
    esm2 = AutoModel.from_pretrained("facebook/esm2_t30_150M_UR50D", cache_dir=cache_directory)
    model = myModel(esm2)
    model = model.to(device)


    criterion = nn.BCELoss()
    optimizer = optim.SGD(model.parameters(), lr=0.001)



    batch_size = 1 

    data_loader = DataLoader(list(zip(X_train, y_train)), batch_size=batch_size, shuffle=True)



    test_data_loader = DataLoader(list(zip(X_test, y_test)), batch_size=batch_size, shuffle=False)

    num_epochs = 60
    for epoch in range(num_epochs):
        total_loss = 0
        
        
        
        for batch_input, batch_labels in data_loader:

            inputs = tokenizer_(batch_input, return_tensors='pt', padding=True, truncation=True)
            inputs = inputs.to(device)

            outputs = model(inputs)
            outputs = outputs.view(-1)
            loss = criterion(outputs.to('cpu'), batch_labels.to(torch.float32))

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            total_loss += loss.item()

        model.eval()  
        test_loss = 0
        correct_predictions = 0
        
        test_labels_com = []
        predict_labels_com = []
        test_outputs_com = []
        
        with torch.no_grad():
            for test_batch_input, test_batch_labels in test_data_loader:
                inputs = tokenizer_(test_batch_input, return_tensors='pt', padding=True, truncation=True)
                inputs = inputs.to(device)


                test_outputs = model(inputs)
                test_outputs = test_outputs.view(-1)
                test_loss += criterion(test_outputs.to('cpu'), test_batch_labels.to(torch.float32)).item()
                predictions = (test_outputs > 0.5).float()  
                correct_predictions += (predictions.to('cpu') == test_batch_labels).sum().item()

    
    
                predict_labels_com.extend(predictions.tolist())
                test_labels_com.extend(test_batch_labels.tolist())
                test_outputs_com.extend(test_outputs.tolist())
            
            
            
        average_test_loss = test_loss / len(test_data_loader)
        accuracy = correct_predictions / len(X_test)


        model.train()
        
        predict_labels_com = np.array(predict_labels_com)
        test_labels_com = np.array(test_labels_com)
        test_outputs_com = np.array(test_outputs_com)
        temp_accuracy,temp_f1,temp_auc,temp_mcc,temp_recall,temp_precision = comp_result(test_labels_com,predict_labels_com,test_outputs_com)
        if temp_accuracy > best_accuracy:
            best_accuracy = temp_accuracy
            best_f1 = temp_f1
            best_auc = temp_auc
            best_mcc = temp_mcc
            best_recall = temp_recall
            best_precision = temp_precision
            save_model = cp.deepcopy(model)
            

        average_train_loss = total_loss / len(data_loader)
        print('Epoch [{}/{}], Train Loss: {:.4f}, Test Loss: {:.4f}, Accuracy: {:.2f}%'.format(
            epoch + 1, num_epochs, average_train_loss, average_test_loss, accuracy * 100))
        
    print('=====================================')
    print('kf',kf_in,"best result:")
    print(best_accuracy)
    print(best_f1)
    print(best_auc)
    print(best_mcc)
    print(best_recall)
    print(best_precision)
    print('=====================================')